## Create Occurrence Data Set including eventDate

### Store GBIF Occurrence Data Set locally

For Virtual Herbarium Germany (BGBM) see <https://doi.org/10.15468/dl.tued2e>. We saved all of it into the local data directory `data/VHde_doi-10.15468-dl.tued2e/`:
- We need `occurrence.txt` as basic data file, which is 1GB large—please download it first (**it will not be in the official GitHub documentation**) or change the code here to read your special input data.

In [13]:
import os, time, pprint

gbif_dataset_path="data/VHde_doi-10.15468-dl.tued2e"

# join file name dynamically for saving results
this_output_tabdata_file=os.path.join(
    gbif_dataset_path, (
        "occurrence_recordedBy_eventDate_occurrenceIDs_%s.tsv" % 
        # '20230726'
        # '20230823'
        time.strftime('%Y%m%d')
    )
)
# use static file name for saving
# this_output_tabdata_file=data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_occurrenceIDs_20230524.tsv

if not os.path.exists(gbif_dataset_path):
    print("Where is the folder of are GBIF occurrence data?", gbif_dataset_path, "not found")
    print("Recommendation is use a subfolder, e.g. “data/VHde_doi-10.15468-dl.tued2e”")
else:
    print("All right: GBIF data found. Results will later be written to", this_output_tabdata_file)

All right: GBIF data found. Results will later be written to data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823.tsv


### Read GBIF Occurrence Data

Get `recordedBy` and `created` of `occurrence.txt` and look into the data first, data columns aso. …

In [7]:
import pandas as pd # to read data

# Reading all at once does not work to read 1GB of data yet
occurrences = pd.read_csv(
    os.path.join(gbif_dataset_path, "occurrence.txt"), sep="\t", low_memory=False,
    nrows=1
)

print(list(occurrences.columns)) # 259 columns

['gbifID', 'abstract', 'accessRights', 'accrualMethod', 'accrualPeriodicity', 'accrualPolicy', 'alternative', 'audience', 'available', 'bibliographicCitation', 'conformsTo', 'contributor', 'coverage', 'created', 'creator', 'date', 'dateAccepted', 'dateCopyrighted', 'dateSubmitted', 'description', 'educationLevel', 'extent', 'format', 'hasFormat', 'hasPart', 'hasVersion', 'identifier', 'instructionalMethod', 'isFormatOf', 'isPartOf', 'isReferencedBy', 'isReplacedBy', 'isRequiredBy', 'isVersionOf', 'issued', 'language', 'license', 'mediator', 'medium', 'modified', 'provenance', 'publisher', 'references', 'relation', 'replaces', 'requires', 'rights', 'rightsHolder', 'source', 'spatial', 'subject', 'tableOfContents', 'temporal', 'title', 'type', 'valid', 'institutionID', 'collectionID', 'datasetID', 'institutionCode', 'collectionCode', 'datasetName', 'ownerInstitutionCode', 'basisOfRecord', 'informationWithheld', 'dataGeneralizations', 'dynamicProperties', 'occurrenceID', 'catalogNumber', 

From previous analysis of `occurrence.txt` and  check for time fields, we know the following fields (and the indexed column):

- `created` (14) 
- `date` (16) 
- `dateAccepted` (17)
- `issued` (35)
- `eventDate` (103)
- `eventTime` (104)
- `startDayOfYear` (105)
- `endDayOfYear` (106)
- `year` (107)
- `month` (108)
- `day` (109)
- `verbatimEventDate` (110)

The whole indexed list of fields can be obtained like so:

```bash
head occurrence.txt -n 1 | sed 's@\t@\n@g' | awk '{print "- " $1 " (" NR ")" }'
# - gbifID (1)
# - abstract (2)
# - accessRights (3)
# - accrualMethod (4)
# - accrualPeriodicity (5)
# - accrualPolicy (6)
# - alternative (7)
# - audience (8)
# - available (9)
# - bibliographicCitation (10)
# …
# - eventType (259)
```

Now get only rows containing any date:

```bash
awk --field-separator='\t'  'BEGIN{OFS="\t"; n_occ=0; any_date="";} { 
  if(NR==1) { print $68, $71, $14, $16, $17, $35, $103, $104, $105, $106, $107, $108, $109, $110 } 
  if (match($68, /^http/)) { n_occ++
    any_date=$14$16$17$35$103$104$105$106$107$108$109$110
    if (length(any_date)) {
      print $68, $71, $14, $16, $17, $35, $103, $104, $105, $106, $107, $108, $109, $110
    }
  }
} ' occurrence.txt  > occurrence-recordedBy-having-any-date-occurrenceID.txt

cat occurrence-recordedBy-having-any-date-occurrenceID.txt  | column --table --separator $'\t' | head
```
… we get:
```
occurrenceID                     recordedBy             created  date  dateAccepted  issued  eventDate            eventTime  startDayOfYear  endDayOfYear  year  month  day  verbatimEventDate
https://je.jacq.org/JE00003430   Zenker,G.                                                   1897-01-01T00:00:00                                           1897              
https://je.jacq.org/JE00003431   Zenker,G.                                                   1897-01-01T00:00:00                                           1897              
https://je.jacq.org/JE00015416   Reverchon,E.                                                1904-07-01T00:00:00                                           1904  7           
https://je.jacq.org/JE00015417   Gaillardot,C.                                               1859-07-25T00:00:00                                           1859  7      25   
https://je.jacq.org/JE00015418   Heldreich,T.H.H. von                                        1891-07-07T00:00:00                                           1891  7      7    
https://je.jacq.org/JE00015419   Heldreich,T.H.H. von                                        1891-07-07T00:00:00                                           1891  7      7    
https://je.jacq.org/JE00015412   Kotschy,C.G.T.                                              1862-07-02T00:00:00                                           1862  7      2    
https://je.jacq.org/JE00015413   Kotschy,C.G.T.                                              1862-07-02T00:00:00                                           1862  7      2    
https://je.jacq.org/JE00015414   Haussknecht,H.K.                                            1865-07-15T00:00:00                                           1865  7      15   
```
… so `eventDate` seems the right date (https://www.gbif.org/data-quality-requirements-occurrences#dcEventDate)

In [8]:
# just see the first rows
occurrences = pd.read_csv(
    os.path.join(gbif_dataset_path, "occurrence.txt"), sep="\t", low_memory=False,
    usecols=["occurrenceID", "recordedBy", "eventDate"],
    nrows=50 # read all data results in memory kill
)
occurrences.head()


,occurrenceID,recordedBy,eventDate
0,NaN,Kurt Harz,NaN
1,NaN,Eugen Erdner,NaN
2,NaN,Alois Zick,NaN
3,NaN,J. Kraenzle,NaN
4,NaN,Hermann Poeverlein,NaN


We follow <https://towardsdatascience.com/tips-and-tricks-for-loading-large-csv-files-into-pandas-dataframes-part-2-5fc02fc4e3ab> and filter for having an occourrenceID.

For large data sets it is better to read it defining a “chunksize” (because otherwise the processor would read all at once and gets stuck):

In [9]:
starttime = time.time()

chunks_occurrences = pd.read_csv(
    os.path.join(gbif_dataset_path, "occurrence.txt"), sep="\t", low_memory=False,
    usecols=["occurrenceID", "recordedBy", "eventDate"],
    chunksize=100000
)

print("read large data as chunk", time.time() - starttime, 'seconds')

def filter_having_occurrenceID(df):
    df = df[df.occurrenceID.notnull()]
    print("filter having occurrenceID: " + str(df.shape))
    # print(df.shape)
    return df

starttime = time.time()
chunk_list = [] # used for storing dataframes
for chunk in chunks_occurrences:
    # each chunk is a dataframe
    # perform data filtering
    filtered_chunk = filter_having_occurrenceID(chunk)
    # Once the data filtering is done, append the filtered chunk to list
    chunk_list.append(filtered_chunk)

# concat all the dfs in the list in
occurrences = pd.concat(chunk_list)

print("process data having only occurrenceID took", time.time() - starttime, ' seconds')
occurrences.dropna(subset=['eventDate'], inplace=True)
occurrences.head()

read large data as chunk 0.00989985466003418 seconds
filter having occurrenceID: (75997, 3)
filter having occurrenceID: (100000, 3)
filter having occurrenceID: (5567, 3)
filter having occurrenceID: (32993, 3)
filter having occurrenceID: (99970, 3)
filter having occurrenceID: (72362, 3)
filter having occurrenceID: (100000, 3)
filter having occurrenceID: (100000, 3)
filter having occurrenceID: (27751, 3)
filter having occurrenceID: (0, 3)
filter having occurrenceID: (0, 3)
process data having only occurrenceID took 18.29196548461914  seconds


,occurrenceID,recordedBy,eventDate
18008,https://je.jacq.org/JE00003430,"Zenker,G.",1897-01-01T00:00:00
18010,https://je.jacq.org/JE00003431,"Zenker,G.",1897-01-01T00:00:00
18011,https://je.jacq.org/JE00015416,"Reverchon,E.",1904-07-01T00:00:00
18012,https://je.jacq.org/JE00015417,"Gaillardot,C.",1859-07-25T00:00:00
18013,https://je.jacq.org/JE00015418,"Heldreich,T.H.H. von",1891-07-07T00:00:00


In [10]:
### Convert to date/time
#
# occurrences.dtypes
#   occurrenceID    object
#   recordedBy      object
#   eventDate       object
#   dtype: object

# df['dates'] = pd.to_datetime(df['dates'], format='%Y%m%d')
# pd.to_datetime("12/29/2020  9:09:37 PM", utc=True)
# pd.to_datetime("1904-07-01T00:00:00", utc=True)

# occurrences['eventDate']= pd.to_datetime(occurrences.eventDate, utc=True) # Out of bounds nanosecond timestamp: 1652-01-01T00:00:00
#  because date nanoseconds range limitations of pandas, see https://stackoverflow.com/a/69507200/1240387
#  work around: use datetime
#  occurrences['eventDate'] = occurrences['eventDate'].apply(lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%S') if type(x)==str else pd.NaT)
# or using pd.Periode(…)
occurrences['eventDate'] = occurrences['eventDate'].apply(lambda x: pd.Period(x, freq='ms'))
occurrences.head()

,occurrenceID,recordedBy,eventDate
18008,https://je.jacq.org/JE00003430,"Zenker,G.",1897-01-01 00:00:00.000
18010,https://je.jacq.org/JE00003431,"Zenker,G.",1897-01-01 00:00:00.000
18011,https://je.jacq.org/JE00015416,"Reverchon,E.",1904-07-01 00:00:00.000
18012,https://je.jacq.org/JE00015417,"Gaillardot,C.",1859-07-25 00:00:00.000
18013,https://je.jacq.org/JE00015418,"Heldreich,T.H.H. von",1891-07-07 00:00:00.000


In [11]:

occurrences.dtypes

occurrenceID       object
recordedBy         object
eventDate       period[L]
dtype: object

In [12]:
# group and aggregate data: 

occurrences_unique=occurrences.groupby(['recordedBy']).agg(
    occurrenceID_count= ('occurrenceID', 'count'), # use count function
    occurrenceID_firstsample=('occurrenceID', lambda x: list(x)[0]) # custom function, to get the first entry
    , eventDate_mean=('eventDate', 'mean')
    , eventDate_min=('eventDate', 'min')
    , eventDate_max=('eventDate', 'max')
).reset_index()

occurrences_unique.head()

,recordedBy,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
0,?,1,http://id.snsb.info/snsb/collection/518649/641...,1874-07-01 00:00:00.000,1874-07-01 00:00:00.000,1874-07-01 00:00:00.000
1,??,1,http://id.snsb.info/snsb/collection/519090/642...,1894-10-28 00:00:00.000,1894-10-28 00:00:00.000,1894-10-28 00:00:00.000
2,A. Azofeifa,2,https://herbarium.bgbm.org/object/B200211416,1998-04-24 12:00:00.000,1998-03-10 00:00:00.000,1998-06-09 00:00:00.000
3,"A. Cano,E.",1,https://herbarium.bgbm.org/object/B100699397,2008-06-05 00:00:00.000,2008-06-05 00:00:00.000,2008-06-05 00:00:00.000
4,A.d.Selmons,1,https://herbarium.bgbm.org/object/B100379213,1917-07-01 00:00:00.000,1917-07-01 00:00:00.000,1917-07-01 00:00:00.000


In [14]:
print("Write these tabbed data into", this_output_tabdata_file)

occurrences_unique.to_csv(this_output_tabdata_file, sep='\t', index=False)

Write these tabbed data into data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823.tsv


## Parsing with dwcagent_bin

Dependency Ruby Gem package <https://libraries.io/rubygems/dwc_agent> has to be installed and Ruby itself.

You can use the ruby script in `bin/agent_parse4tsv.rb` and change the file(s) for input and output:
```bash
cd bin
ruby agent_parse4tsv.rb --help # display usage and help of the script (use --develop to keep also original input data in parsed results)

ruby agent_parse4tsv.rb  --develop \
  --input  ../data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823.tsv \
  --output ../data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823_parsed.tsv

# or if you want to measure how fast it parses use
time ruby agent_parse4tsv.rb \
  --input  ../data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823.tsv \
  --output ../data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823_parsed.tsv
# real    0m39,869s
# user    0m21,486s
# sys     0m11,357s
```

And look into the first data lines, e.g.
```bash
head ../data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230726_parsed.tsv | column --table --separator $'\t'
# … you could get something like:
# family     given  suffix  particle  dropping_particle  nick  appellation  title  occurrenceID_count  occurrenceID_firstsample                                  eventDate_mean           eventDate_min            eventDate_max
# Azofeifa   A.                                                                    2                   https://herbarium.bgbm.org/object/B200211416              1998-04-24 12:00:00.000  1998-03-10 00:00:00.000  1998-06-09 00:00:00.000
# A. Cano    E.                                                                    1                   https://herbarium.bgbm.org/object/B100699397              2008-06-05 00:00:00.000  2008-06-05 00:00:00.000  2008-06-05 00:00:00.000
# Selmons    Ad                                                                    1                   https://herbarium.bgbm.org/object/B100379213              1917-07-01 00:00:00.000  1917-07-01 00:00:00.000  1917-07-01 00:00:00.000
# Aaronsohn  A.                                                                    1                   https://herbarium.bgbm.org/object/B100379341              1908-06-20 00:00:00.000  1908-06-20 00:00:00.000  1908-06-20 00:00:00.000
# Ani        H.             Abbas al                                               1                   http://id.snsb.info/snsb/collection/462713/563871/241553  1964-11-18 00:00:00.000  1964-11-18 00:00:00.000  1964-11-18 00:00:00.000
# Abbe       L.B.                                                                  1                   https://herbarium.bgbm.org/object/BGT0003826              1960-03-18 00:00:00.000  1960-03-18 00:00:00.000  1960-03-18 00:00:00.000
# Abbe       E.C.                                                                  1                   https://herbarium.bgbm.org/object/BGT0003826              1960-03-18 00:00:00.000  1960-03-18 00:00:00.000  1960-03-18 00:00:00.000
```